In [1]:
import time

import optuna
from parameters import Parameters
from model import *
import example_parameters

In [2]:
def normalize(array):
    max_ = max(array)
    return [x / max_ for x in array]

simulation_len = 1000
epsilon = .1

params = example_parameters.elite_equilibrium

# computation
def compute(parameters, simulation_len, epsilon):
    xc, xe, y, w, am, aM, bc, be, s, gamma, lambda_, kappa, delta, rho = parameters.unpacked()

    for i in range(int(simulation_len / epsilon)):
        xc, xe, y, w = step(xc, xe, y, w, am, aM, bc, be, s, gamma, lambda_, kappa, delta, rho, epsilon)

    return xc, xe, y, w


def objective(trial: optuna.Trial):
    parameters = example_parameters.elite_equilibrium
    xc, xe, y, w, am, aM, bc, be, s, gamma, lambda_, kappa, delta, rho = parameters.unpacked()

    am = trial.suggest_float("am", parameters.am / 1.1, parameters.am * 1.1)
    aM = trial.suggest_float("aM", parameters.aM / 1.1, parameters.aM * 1.1)
    bc = trial.suggest_float("bc", parameters.bc / 1.1, parameters.bc * 1.1)
    be = trial.suggest_float("be", parameters.be / 1.1, parameters.be * 1.1)
    s = trial.suggest_float("s", parameters.s / 1.1, parameters.s * 1.1)
    rho = trial.suggest_float("rho", parameters.rho / 1.1, parameters.rho * 1.1)
    gamma = trial.suggest_float("gamma", parameters.gamma / 1.1, parameters.gamma * 1.1)
    lambda_ = trial.suggest_float("lambda_", parameters.lambda_ / 1.1, parameters.lambda_ * 1.1)
    # kappa = trial.suggest_float("kappa", 1, 100)
    kappa = trial.suggest_float("kappa", 1, 2)
    delta = trial.suggest_float("delta", parameters.delta / 1.1, parameters.delta * 1.1)
    xc = trial.suggest_float("xc", parameters.xc / 1.1, parameters.xc * 1.1)
    xe = trial.suggest_float("xe", parameters.xe / 1.1, parameters.xe * 1.1)
    y = trial.suggest_float("y", parameters.y / 1.1, parameters.y * 1.1)
    w = trial.suggest_float("w", parameters.w / 1.1, parameters.w * 1.1)

    return compute(Parameters(xc, xe, y, w, am, aM, bc, be, s, gamma, lambda_, kappa, delta, rho, 1000, 0.1), 1000, 0.1)


study = optuna.create_study(directions=["maximize","maximize","maximize","maximize"])
start = time.time()
study.optimize(objective, n_trials=300, timeout=300, show_progress_bar=True)

print(f"Finished in {time.time() - start}")
print("Number of finished trials: ", len(study.trials))


[I 2022-11-02 17:41:03,660] A new study created in memory with name: no-name-65a5fc6e-1f16-43dc-bb20-cede7059c93e
C:\Users\Pawel\Documents\AGH\ISZ\isz_python\venv\Lib\site-packages\optuna\progress_bar.py:47: ExperimentalWarning: Progress bar is experimental (supported from v1.2.0). The interface can change in the future.
  self._init_valid()


  0%|          | 0/300 [00:00<?, ?it/s]

[I 2022-11-02 17:41:03,698] Trial 0 finished with values: [21335.60631889436, -3.806474544125792e+37, 0.0, 4.732763075993833e-298] and parameters: {'am': 0.0103548870292474, 'aM': 0.07236297901214325, 'bc': 0.029096558272039056, 'be': 0.027533569538886277, 's': 0.0005155333276747047, 'rho': 0.005077846349304185, 'gamma': 0.009373221259927617, 'lambda_': 100.98756111938187, 'kappa': 1.1391537384127677, 'delta': 1.0352792290063677e-05, 'xc': 103.87340143302545, 'xe': 0.9177466614532586, 'y': 106.64043285982571, 'w': 0.0}. 
[I 2022-11-02 17:41:03,713] Trial 1 finished with values: [9185.467749263635, -9.307668502504659e+35, 0.0, 8.54454048388843e-299] and parameters: {'am': 0.009942220556584107, 'aM': 0.06888500798833244, 'bc': 0.030383850134675285, 'be': 0.028337776740727575, 's': 0.0005404891939511048, 'rho': 0.005086469036030884, 'gamma': 0.010028704083541602, 'lambda_': 103.05227531226156, 'kappa': 1.178522170195688, 'delta': 9.466582115652021e-06, 'xc': 94.56169047009891, 'xe': 1.032

In [ ]:
# optuna.visualization.plot_pareto_front(study, target_names=["xc", "xe", "y"])

In [3]:
optuna.visualization.plot_param_importances(
    study, target=lambda t: t.values[0], target_name="flops"
)

ZeroDivisionError: Weights sum to zero, can't be normalized

In [4]:
print(f"Number of trials on the Pareto front: {len(study.best_trials)}")

trial_with_highest_accuracy = max(study.best_trials, key=lambda t: t.values[3])
print(f"Trial with highest accuracy: ")
print(f"\tnumber: {trial_with_highest_accuracy.number}")
print(f"\tparams: {trial_with_highest_accuracy.params}")
print(f"\tvalues: {trial_with_highest_accuracy.values}")

Number of trials on the Pareto front: 66
Trial with highest accuracy: 
	number: 260
	params: {'am': 0.010827743455109303, 'aM': 0.06674688780208343, 'bc': 0.027596406761462952, 'be': 0.03282185896580114, 's': 0.0004852558290170412, 'rho': 0.005384243840911763, 'gamma': 0.009488390244385803, 'lambda_': 103.43601074756884, 'kappa': 1.8835706782505013, 'delta': 1.0410117806814998e-05, 'xc': 99.07122948829166, 'xe': 1.0999394742343038, 'y': 94.039934065497, 'w': 0.0}
	values: [2224.464198727425, -9.160377493204764e+34, 0.0, 1.0356859054176267e-291]
